In [2]:
!git clone https://github.com/allanzelener/yad2k.git

Cloning into 'yad2k'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 2.35 MiB | 4.43 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [10]:
import os
import tensorflow as tf
import numpy
import pandas
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model

import argparse#
import PIL#
import scipy.io#
import scipy.misc
from yad2k.yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

In [9]:
# Input shape are images of size (b, 608, 608, 3)
# Output shape should look (m, 19, 19, 5, 85) 85: (pc,bx,by,bh,bw,c1,...,c80) 
# For each grid in 19x19 midpoint of an object and anchor box is assigned
# Then use Non-Max surpression to reduce output to smaller # of detected objects
  # Get rid of boxes with a low probability
  # Choose only one box when several boxes overlap with each other and detect the same object.


In [7]:
      # Filtering by thresholding over the class scores
def yolo_filter_boxes(box_object_presence, boxes, box_class_probs, threshold = .6):
  #box_object_presence, -- tensor of shape (19, 19, 5, 1)
  #boxes -- tensor of shape (19, 19, 5, 4)
  #box_class_probs -- tensor of shape (19, 19, 5, 80) contains class probabilities
  # for each of the 80 classes for each of the 5 boxes per cell.

  # Step 1: Compute box scores(probabilities)
    box_scores = box_object_presence * box_class_probs #(19*19, 5, 80)
    
    # Step 2: Find the index of the class with the maximum box score, the corresponding box score
    box_classes =  K.argmax(box_scores,axis = -1)
    box_class_scores = K.max(box_scores, axis = -1)
    
    # Step 3: Create a filtering mask based on "box_class_scores" by using "threshold". The mask should have the
    # same dimension as box_class_scores, and be True for the boxes you want to keep (with probability >= threshold)
    filtering_mask = box_class_scores >= threshold
    
    # Step 4: Apply the mask to scores, boxes and classes to remove useless boxes
    scores = tf.boolean_mask(box_class_scores, filtering_mask, name = 'boolean_mask')  # Keeping a score of boxes
    boxes = tf.boolean_mask(boxes, filtering_mask,  name = 'boolean_mask')             # Keeping only appropriate boxes
    classes = tf.boolean_mask(box_classes, filtering_mask, name = 'boolean_mask')      # Keeps index of the class

    return scores, boxes, classes

In [8]:
        # Implement Intersection over Union (measuring accuracy)
def iou(box1, box2):

    # (x1, y1) upper left, (x2, y2) lower right corners of box
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # Calculate the (yi1, xi1, yi2, xi2) coordinates of the intersection of box1 and box2.
    xi1 = np.max([box1[0], box2[0]]) #x-coordinate is closer to the right
    yi1 = np.max([box1[1], box2[1]]) #y-coordinate that is closer to the bottom
    xi2 = np.min([box1[2], box2[2]]) #x-coordinate is closer to the left
    yi2 = np.min([box1[3], box2[3]]) #y-coordinate that is closer to the top
    inter_area=max(xi2-xi1,0)*max(yi2-yi1,0)  # Area = width * height

    #  Union(A,B) = A + B - Inter(A,B)
    box1_area = (box1[3] - box1[1]) * (box1[2] - box1[0])
    box2_area = (box2[3] - box2[1]) * (box2[2] - box2[0])
    union_area = box1_area + box2_area - inter_area
    
    # compute the IoU
    iou = inter_area / union_area

    return iou

In [11]:
      # Implementing Non-Max
         # Select the box that has the highest score.
         # Compute the overlap of this box with all other boxes, and remove boxes that overlap significantly (iou >= iou_threshold).
         #Go back to step 1 and iterate until there are no more boxes with a lower score than the currently selected box.
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor

    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor , iou_threshold = 0.5)

    scores = K.gather(scores,nms_indices)
    boxes = K.gather(boxes,nms_indices)
    classes = K.gather(classes,nms_indices)


    return scores, boxes, classes